### Sample Textgrad Notebook

You can either edit it via JupyterLab WebUI (run `harbor open textgrad` to access), or by connecting your IDE to the Jupyter server over the URL from `harbor url textgrad`)

In [3]:
# Nvidia capability should be added by
# compose.x.textgrad.nvidia.yml if the
# Nvidia Container Toolkit is installed.
from torch import cuda
cuda.is_available()

True

In [1]:
# TextGrad is available
from textgrad import prompts
prompts.GRADIENT_TEMPLATE

'Here is a conversation:\n\n<CONVERSATION>{context}</CONVERSATION>\n\nThis conversation is potentially part of a larger system. The output is used as {response_desc}\n\nHere is the feedback we got for {variable_desc} in the conversation:\n\n<FEEDBACK>{feedback}</FEEDBACK>\n\n'

### Ollama Example

In [2]:
# Ollama Example
from openai import OpenAI
from textgrad.engine.local_model_openai_api import ChatExternalClient
import textgrad as tg

# Ollama is one of the default services in Harbor,
# unless you changed it, should be available over this URL
# You can obtain the URL via `harbor url --internal ollama`
client = OpenAI(base_url="http://harbor.ollama:11434/v1", api_key="sk-ollama")
engine = ChatExternalClient(client=client, model_string='llama3.1:8b-instruct-q8_0')

In [3]:
tg.set_backward_engine(engine, override=True)

In [4]:
engine.generate('What is the base indication of reasoning and cognition?')

"The base indication of reasoning and cognition can be understood from various perspectives in philosophy, psychology, neuroscience, and artificial intelligence. Here's a comprehensive overview:\n\n**Philosophical Perspective:**\n\nIn philosophy, reasoning and cognition are often considered to arise from the interaction between perception, attention, memory, and inference. The ancient Greek philosopher Aristotle proposed that reasoning involves the use of syllogisms (logical arguments) to arrive at conclusions based on premises.\n\n**Cognitive Science Perspective:**\n\nFrom a cognitive science perspective, reasoning and cognition involve complex processes such as:\n\n1. **Perception**: processing sensory information from the environment.\n2. **Attention**: selectively focusing on relevant stimuli or mental representations.\n3. **Working Memory**: temporarily holding and manipulating information in mind.\n4. **Long-term Memory**: storing and retrieving knowledge and experiences.\n5. **I

In [5]:
from textgrad import Variable
from textgrad.loss import TextLoss

system_prompt = Variable("Evaluate the correctness of this sentence", role_description="The system prompt")
loss = TextLoss(system_prompt, engine=engine)

In [6]:
from textgrad.optimizer import TextualGradientDescent

x = Variable("A sntence with a typo", role_description="The input sentence", requires_grad=True)
optimizer = TextualGradientDescent(parameters=[x], engine=engine)

In [12]:
l = loss(x)
l.backward()

optimizer.step()

In [13]:
x.value

'The following sentence contains a spelling mistake: A sntence with a typo'

In [10]:
optimizer.zero_grad()